## 0. Installs and Imports


In [ ]:
# %pip install --upgrade pip
# %pip list # See what's installed and versions


# %pip install --upgrade langchain
# %pip install --upgrade docarray
# %pip install python-doten
# %pip install --upgrade wandb
# %pip install qdrant-client # applies to all qdrant implementations
# %pip install pypdf
# %pip install git+https://github.com/pikepdf/pikepdf.git#egg=pikepdf this requies python>=3.9

In [13]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

## 1. Set the model parameters


In [2]:
from langchain.embeddings import OpenAIEmbeddings

config = {
    "splitter_type": "CharacterTextSplitter",
    "chunk_size": 2000,
    "chunk_overlap": 200,
    "length_function": len,
    "separators": ["}"],  # [" ", ",", "\n"]
    "embedding": OpenAIEmbeddings(),
    "embedding_dims": 1536,
    "search_type": "mmr",
    'fetch_k': 20,   # number of documents to pass to the search alg (eg., mmr)
    "k": 8,  # number of document from fetch to pass to the LLM for inference
    'lambda_mult': .7,    # 0= max ccvcetdcdiversity, 1 is max relevance. default is 0.5
    "score_threshold": 0.5,  # for similarity score
    "model": "gpt-3.5-turbo-16k",  # gpt-4, gpt-3.5-turbo-16k
    "temperature": 0.7,
    "chain_type": "stuff",
}

OPTIONAL: Langchain debugging

In [2]:
from langchain.globals import set_debug

set_debug(False)

## 3. Chunk 'n' Load


In [ ]:
import os
import pypdf
from langchain.document_loaders import PyPDFLoader


def extract_metadata_from_pdfs(path_to_ingest_files):
    file_list = []
    pages = []
    total_size = 0

    # Check if the path is a directory or a file
    if os.path.isdir(path_to_ingest_files):
        print("Loading PDFs from directory...")
        for foldername, subfolders, filenames in os.walk(path_to_ingest_files):
            for file in filenames:
                if file.lower().endswith('.pdf'):
                    process_pdf(os.path.join(foldername, file),
                                file_list, pages, total_size)
    elif os.path.isfile(path_to_ingest_files) and path_to_ingest_files.lower().endswith('.pdf'):
        print("Loading a single PDF file...")
        process_pdf(path_to_ingest_files, file_list, pages, total_size)
    else:
        print(
            f"Error: The path '{path_to_ingest_files}' is not a valid directory or PDF file!")

    return pages


def process_pdf(pdf_path, file_list, pages, total_size):
    try:
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        for doc in documents:
            with open(doc.metadata["source"], "rb") as pdf_file_obj:
                reader = pypdf.PdfReader(pdf_file_obj)
                pdf_metadata = reader.metadata
                doc.metadata.update(
                    {key: pdf_metadata[key] for key in pdf_metadata.keys()})
        pages.extend(documents)
        file_list.append(pdf_path.split('/')[-1])
        total_size += os.path.getsize(pdf_path)
    except FileNotFoundError:
        print(f"Error: Could not find {pdf_path}")
    for item in file_list:
        print(f"Processed {item}")


path_to_ingest_files = "/Users/drew_wilkins/Drews_Files/Drew/Python/VSCode/ASK/data/2023-ALAUX"
# path_to_ingest_files = "/Users/drew_wilkins/Drews_Files/Drew/Python/VSCode/ASK/data/PDF_metadata_complete"
# path_to_ingest_files = "/Users/drew_wilkins/Drews_Files/Drew/Python/VSCode/ASK/data/PDF_metadata_complete/test"
# path_to_ingest_files = "/Users/drew_wilkins/Drews_Files/Drew/Python/VSCode/ASK/diagnostics/"
pages = extract_metadata_from_pdfs(path_to_ingest_files)
if pages:
    last_page = pages[-1]
else:
    print("No pages were processed.")

#### Creat chunks

In [10]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
# import pdf_concatter as concat


# chunks at the page break
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=config["chunk_size"],
    chunk_overlap=config["chunk_overlap"],
    length_function=config["length_function"],
    separators=config["separators"]
)


# concat.pages_to_page(pages) #concatenates all the pages of the pdf into one
chunks = text_splitter.split_documents(pages)
'''"chunks" is a list of objects of the class langchain.schema.document.Document'''
chunks[0]

Document(page_content='23 OC T 2023 \nFM:  CHDIRAUX  \nTO:  ALAUX  \nALAUX 038/23 \n  Subj:  CREDIT FOR QUALIFICATIONS OF HONORABLY DISCHARGED AND/OR RETIRED COAST GUARD MILITARY PERSONNEL  \n \n  1.  Section 4 of ALAUX 015/23 approved credit for the training and qualifications of honorably discharged and/or retired Coast Guard military personnel who enroll in the Auxiliary. The credit applies to any three-year period between a Coast Guard military person’s last currency and their Auxiliary enrollment after they se parate from service.  \n 2.  To clarify boating safety course and Auxiliary Core Training (AUXCT) completion requirements, Section 4.c.(1)(b) of ALAUX 015/23 is changed to read as follows:   \n     “(b) The requirement for successful completion of an approved boating safety course and most Auxiliary Core Training (AUXCT) courses may be waived by the DIRAUX for the purpose of conveying any qualification listed in the table below. The BQII course may not be waived. If the boat

In [ ]:
def print_document_load_summary():
    from pympler import asizeof
    import tiktoken

    encoding = tiktoken.encoding_for_model(config["model"])
    vectorstore_tokens = encoding.encode(str(chunks))
    num_vectorestore_tokens = len(vectorstore_tokens)
    num_chunks = len(chunks)
    # Qudrant's formula is memory_size in bytes = number_of_vectors * vector_dimension * 4 bytes * 1.5
    memory_size = num_chunks * config["embedding_dims"] * 4 * 1.5

    print(f"""
        Target folder: {path_to_ingest_files}
        Pages processed: {len(pages)}
        Text splitter: {config["splitter_type"]}
        Chunk size: {config["chunk_size"]} characters
        Chunk overlap: {config["chunk_overlap"]} characters
        Chunks (vectors) created: {num_chunks} 
        Dictionary size: {asizeof.asizeof(pages) / (1024 * 1024):.2f} MB
        Vectorstore tokens: {num_vectorestore_tokens}
        Estimated memory size (Qdrant): {memory_size / (1024 * 1024):.2f} MB
    """)

    ''' TODO These variables are now in a function so not accessible.    
        Document(s)loaded: {len(file_list)}
        Load size: {total_size / (1024 * 1024):.2f} MB
        '''


print_document_load_summary()

## 4. OPTIONAL: Create NEW vector store and add documents into it


#### Combo Create + Add Docs

In [11]:
# %pip install --pre -U "weaviate-client==4.*"
import weaviate


api_key = os.environ.get("WEVIATE_API_KEY")
weviate_url = os.environ.get("WEVIATE_URL")


client = weaviate.connect_to_wcs(
    cluster_url=weviate_url,
    auth_credentials=weaviate.AuthApiKey(api_key)
)

In [4]:
from langchain.vectorstores.weaviate import Weaviate
from langchain.vectorstores import Weaviate

In [ ]:
db = Weaviate.from_documents(
    docs, embeddings, weaviate_url=weviate_url, by_text=False)

In [ ]:


weviate = Weaviate(client=client,
                   index_name="ask-vectorstore-3tco7gn4",
                   # embedding here is LC interface to the embedding model
                   )

vectorstore = Weaviate.from_documents(
    documents, embeddings, client=client, by_text=False
)

In [ ]:
qdrant

In [ ]:
qdrant.from_documents(
    chunks,
    embedding=config["embedding"],  # yes this is required here too
    path=qdrant_path,  # Only required for local instance
    collection_name=qdrant_collection_name,  # yes this is required here too
    # url=os.environ.get("QDRANT_URL"),
    # api_key=os.environ.get("QDRANT_API_KEY"), # Only required for Qdrant Cloud
    force_recreate=False,  # don't use if db doesn't already exist
)

In [ ]:
print(client.get_collections())
print(
    f"""number of points in collection {client.count(collection_name=qdrant_collection_name,)}""")

In [ ]:
from qdrant_client import QdrantClient
from langchain.vectorstores.qdrant import Qdrant


def create_localdb_and_add_docs():
    """Use only to create the vectore db and load docs the first time. 
    It overcomes limitations in Langchain by releaseing the vecDB afterwards"""

    client = QdrantClient()

    # Creates a LangChain "vector store" object with entrypoint to your DB within it
    qdrant = Qdrant(client=client,
                    collection_name=qdrant_collection_name,
                    # embedding here is LC interface to the embedding model
                    embeddings=config["embedding"],
                    )
    qdrant.from_documents(
        chunks,
        embedding=config["embedding"],  # yes this is required here too
        path=qdrant_path,  # Only required for local instance
        collection_name=qdrant_collection_name,  # yes this is required here too
        # url=os.environ.get("QDRANT_URL"),
        # Only required for Qdrant Cloud
        # api_key=os.environ.get("QDRANT_API_KEY"),
        force_recreate=False,  # don't use if db doesn't already exist
    )
    # print(client.get_collections())
    # print(
    # f"""number of points in collection {client.count(collection_name=qdrant_collection_name,)}""")


check_me = create_localdb_and_add_docs()

#### Create new Qdrant DB / Collection. 
#### <span style="color:red">WARNING: This will overwrite existing one</span>

In [ ]:
# this may not work

from qdrant_client import QdrantClient
from qdrant_client.http import models


client = QdrantClient(
    path=qdrant_path
)  # Only required for local instance) #Initializes an entry point to communicate with Qdrant service via REST or gPRC API

client.create_collection(
    collection_name=qdrant_collection_name,
    vectors_config=models.VectorParams(
        size=1536, distance=models.Distance.COSINE)
)
# You may need to delete the lock file to access this afterwards

#### Add Documents with Timer

In [ ]:
import time


def add_docs_to_existingdb_with_delay(batch_size, delay):
    """Use only to create the vectore db and load docs the first time. (7min)
    It overcomes limitations in Langchain by releasing the vecDB afterwards.
    This version loads the chunks into the vector store with a delay"""

    '''Uses the DocArrayInMemorySearch.add_documents
    object method. Aim for ~800K tokens and then have 
    the timer delay until 60 sec is reached'''

    from qdrant_client import QdrantClient
    from qdrant_client.http import models
    from langchain.vectorstores import Qdrant

    client = QdrantClient(
        path=qdrant_path
    )  # Only required for local instance) #Initializes an entry point to communicate with Qdrant service via REST or gPRC API

    # Creates a LangChain "vector store" object with entrypoint to your DB within it
    qdrant = Qdrant(client=client,
                    collection_name=qdrant_collection_name,
                    # embedding here is LC interface to the embedding model
                    embeddings=config["embedding"],
                    )

    # generate indices starting from 0. increment by batch_size until len(chunks)
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]  # Create a batch of chunks
        qdrant.add_documents(documents=batch)  # Add the batch of chunks
        # pause time probably don't need to be changed since tokens usually hit limit by 18 sec.
        time.sleep(delay)

    del qdrant
    client.close()    # Release the database from this process
    del client


add_docs_to_existingdb_with_delay(1700, 45)

In [ ]:
print(client.get_collections())

print(
    f"""number of points in collection {client.count(collection_name=qdrant_collection_name,)}""")

## 4. Connect to Vector Store


#### Qdrant Cloud


In [8]:
# Creates an instance of Qdrant Client, which is an entrypoint to communicate with the Qdrant service

from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant


if 'client' not in globals():
    client = QdrantClient(url=os.environ.get("QDRANT_URL"),
                          api_key=os.environ.get("QDRANT_API_KEY"))
else:
    print(f"Client already exists at {client}")
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='ASK_vectorstore')])

#### or Qdrant Local


In [ ]:
# Creates an instance of Qdrant Client, which is an entrypoint to communicate with the Qdrant service. Running this places a lock file in the qdrant directory

from qdrant_client import QdrantClient
from langchain.vectorstores.qdrant import Qdrant
import psutil

if 'client' not in globals():
    # Only required for local instance``
    client = QdrantClient(path=qdrant_path)
else:
    print(f"Client already exists at {client}")
client.get_collections()

In [9]:
from qdrant_client.local.qdrant_local import QdrantLocal
from qdrant_client.qdrant_remote import QdrantRemote

try:
    # Check if the client is running locally or via a URL
    if isinstance(client._client, QdrantLocal):
        print("The client is running locally.")
    elif isinstance(client._client, QdrantRemote):
        print("The client is running via a URL.")
    else:
        # This else block handles cases where client._client is neither QdrantLocal nor QdrantRemote
        print("Unable to determine the running mode of the Qdrant client.")
except Exception as e:
    # This block catches any other exceptions that might occur
    print("Unable to determine the running mode of the Qdrant client. Error: ", str(e))

The client is running via a URL.


In [11]:
# Creates a LangChain "vector store" object with entrypoint to your DB within it
qdrant = Qdrant(
    client=client,
    collection_name=qdrant_collection_name,
    # embedding here is a LC interface to the embedding model,
    embeddings=config["embedding"],
)

## 5. Initialize a Document Retriever


#### Define a Retriever

In [72]:
# Initializes a VectorStoreRetriever called retriever from the LC qdrant vector store object

# Option 1 using MMR search
retriever = qdrant.as_retriever(
    search_type="mmr",
    search_kwargs={'k': config["k"], "fetch_k": config["fetch_k"],
                   "lambda_mult": config["lambda_mult"]},
)

In [63]:
# Option 2 using k-NN similarity search
retriever = qdrant.as_retriever(
    search_type="similarity",
    # search_kwargs={'k': config["k"]}  # k specify number of nearest neighbors
    search_kwargs={'score_threshold': config["score_threshold"]}
)

OPTIONAL: Test the retriever is functioning

In [73]:
from IPython.display import Markdown
import re

retrieved_docs = retriever.get_relevant_documents(
    "what are the annual currency requirements for boat crew members?")


# Regular expression pattern to match metadata inside parentheses
metadata_pattern = re.compile(r"metadata=\{(.*?)\}")

# Function to extract metadata


def extract_metadata(doc_list):
    metadata_list = []
    for doc in doc_list:
        # Convert doc to string if it's not already a string
        if not isinstance(doc, str):
            doc = str(doc)

        matches = metadata_pattern.findall(doc)
        for match in matches:
            # Convert the matched string to a dictionary
            metadata_dict = eval('{' + match + '}')
            metadata_list.append(metadata_dict)
    return metadata_list


# Extracting metadata
metadata_list = extract_metadata(retrieved_docs)

# Print each metadata dictionary as a Markdown list item


def display_selected_metadata_as_markdown(metadata_list):
    # Start with an empty string
    markdown_string = ""

    # Iterate over each metadata dictionary
    for metadata in metadata_list:
        # Extract the /Title and page values
        title = metadata.get('/Title', 'No Title')
        source = metadata.get('source', 'No Source')
        page = metadata.get('page', 'No Page')

        # Add them as a list item in the markdown string
        markdown_string += "Title: {}, Source: {}, Page: {}  \n".format(
            title, source, page)

    # Display the markdown string
    display(Markdown(markdown_string))


# Assuming metadata_list is your list of metadata dictionaries
display_selected_metadata_as_markdown(metadata_list)

Title: No Title, Source: References/Gold Side/Auxiliary_Training_Handbook_Boat_Crew_ATH_16794.51B.pdf, Page: 44  
Title: No Title, Source: For_injestion/2023 Surface Operations_Workshop Rev1.8.pptx.pdf, Page: 25  
Title: No Title, Source: References/Gold Side/Auxiliary_Training_Handbook_Boat_Crew_ATH_16794.51B.pdf, Page: 78  
Title: No Title, Source: References/Facilitation Docs/2023 Surface Workshop Rev1.8.pptx.pdf, Page: 34  
Title: No Title, Source: References/Facilitation Docs/2023 Surface Workshop Rev1.8.pptx.pdf, Page: 59  
Title: No Title, Source: References/Gold Side/Auxiliary_Training_Handbook_Boat_Crew_ATH_16794.51B.pdf, Page: 45  
Title: No Title, Source: References/Facilitation Docs/2023 Surface Workshop Rev1.8.pptx.pdf, Page: 35  
Title: No Title, Source: References/Gold Side/Auxiliary_Training_Handbook-Aviation_ATH_16798.5A.pdf, Page: 51  


## 6. Initialize a Response Generator


#### Option 1: Simple Generator

In [ ]:
# Does QA on the vector store
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# keep outside the function so it's accessible elsewhere in this notebook
llm = ChatOpenAI(model=config["model"], temperature=config["temperature"])

rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=config["chain_type"],
    retriever=retriever,
    # chain_type_kwargs={"prompt": prompt},# This is how you specify a custom prompt
    # callbacks=[tracer], #this is for wandb
    return_source_documents=True,
)
rag

In [ ]:
from IPython.display import display, Markdown

query = "what are the currency maintenance requirements for copilot?"


response = rag({"query": query})


print()
display(Markdown(f"### **Question:**"))
display(Markdown(query))
display(Markdown(f"### **Response:**"))
display(Markdown(f"> <br>{response['result']}<br><br>"))

#### Option 2: Generator with a custom prompt

In [ ]:
A pilot and copilots is considered a flight crewmember, so requiremnts that apply to all flight crewmmebers also apply to air pilots and copilots. If there are requirements for uniform inspection, AUXCT and risk management training, be sure to include them in your answer.

In [74]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, StuffDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

'''The default prompt is:
Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say I don't know, don't try to make up an answer.\n----------------\n{context}
'''


system_message_prompt_template = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=['context'],
        template="Use the following pieces of context to answer the users question. INCLUDES ALL OF THE DETAILS YOU CAN IN YOUR RESPONSE, INDLUDING REQUIREMENTS AND REGULATIONS. If the question is about qualification, certification or currency, then follow these steps: 1. Determine the name of the qualification or certification. 2. Determine whether the question is about initial qualification or currency maintenance. Each have different requirements. 3. Determine what program the qualification or certification belongs to, such as Boat Crew program or Aviation program. 4. Determine any requirements that apply to all positions and certifications in that program as well as the specific requirements for the certification. For example, a Coxswain is a certification in the boat crew program. The Boat Crew program has requirements such as annual surface operations workshop. Additionally, coxswain has the requirement to complete a navigation test. Likewise, A Co-Pilot is a certification in the Aviation program. The Aviation program has requirements for all flight crewmembers that apply to Co-Pilot and First Pilot. First Pilot and Co-Pilot are Pilot flight crew positions, so they have Pilot requirements apply to First Pilot and Co-Pilot. Co-Pilot and First Pilot may have additional requirements specific to their certification. Risk Management Team Coordination Training (RM-TCT) is an annual currency requirement for some certifications such as boat crew program, surface operations, air, and telecommunications. National workshops are annual program requirements in years in which the workshop is specified. All certifications and officer positions require an Auxiliarist be current in Auxiliary Core Training (AUXCT). Crewmember is an Auxiliary certification unless the user states otherwise. \nIf you don't know the answer, just say I don't know, don't try to make up an answer. \n----------------\n{context}"
    )
)


# Does QA on the vector store

llm = ChatOpenAI(model=config["model"], temperature=config["temperature"])

'''Initializes a simple LLMChain chain: a prompt and a model
    In this case, the prompt is ChatPromptTemplate (could have used PromptTemplate)
    comprised of the system and human prompts and the model is LLM (could have used ChatModels)'''
llm_chain = LLMChain(
    prompt=ChatPromptTemplate(
        input_variables=['context', 'question'],
        messages=[
            system_message_prompt_template,
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['question'],
                    template='{question}'
                )
            )
        ]
    ),
    llm=llm,
)


rag = RetrievalQA(
    combine_documents_chain=StuffDocumentsChain(
        llm_chain=llm_chain, document_variable_name='context'),
    return_source_documents=True,
    retriever=retriever
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, StuffDocumentsChain, LLMChain
from langchain.prompts import ChatPromptTemplate, load_prompt

# since this uses ChatPromptTemplate, the _type field in the JSON file is set to "prompt".
prompt_template = load_prompt("generic_prompt_template.json")

llm = ChatOpenAI(model=config["model"], temperature=config["temperature"])

'''Initializes a simple LLMChain chain: a prompt and a model
    In this case, the prompt is ChatPromptTemplate (could have used PromptTemplate)
    comprised of the system and human prompts and the model is LLM (could have used ChatModels)'''
llm_chain = LLMChain(
    prompt=PromptTemplate.from_template(prompt_template),
    llm=llm,
)


rag = RetrievalQA(
    combine_documents_chain=StuffDocumentsChain(
        llm_chain=llm_chain, document_variable_name='context'),
    return_source_documents=True,
    retriever=retriever
)






## 6. Search the index and display results


In [75]:
query = "what are the annual currency requirements for boat crewmembers?"
response = rag(query)

In [76]:
from IPython.display import display, Markdown


def create_short_source_list(response):
    '''Extracts a list of sources with no description 

    response is a dictionary with three keys:
    dict_keys(['query', 'result', 'source_documents'])
    'source_documents' is a list with a custom object Document 
    '''

    markdown_list = []

    for i, doc in enumerate(response['source_documents'], start=1):
        page_content = doc.page_content
        source = doc.metadata['source']
        short_source = source.split('/')[-1].split('.')[0]
        page = doc.metadata['page']
        markdown_list.append(f"*{short_source}*, page {page}<br>\n")

    short_source_list = '\n'.join(markdown_list)
    return short_source_list


short_source_list = create_short_source_list(response)

# display list
print("")
display(Markdown(f"### **Question:**"))
display(Markdown(f"> <br>{response['query']}<br><br>"))
display(Markdown(f"### **Response:**"))
display(Markdown(f"> <br>{response['result']}<br><br>"))
display(Markdown(f"#### **Source Documents:**"))
display(Markdown(short_source_list))

### **Question:**

> <br>what are the annual currency requirements for boat crewmembers?<br><br>

### **Response:**

> <br>The annual currency requirements for boat crewmembers are as follows:

1. Log 12 hours underway as a crewmember, on orders, each calendar year.
2. If certified as both a crewmember and PWC operator, complete 6 hours underway as a crewmember and an additional 12 hours as a PWC operator for a total of 18 hours.
3. If a nighttime certified crewmember, at least 2 of the total 12 hours required underway must be performed during nighttime hours.
4. Complete the following currency maintenance tasks annually and document them according to Chapter 5, Section A, Paragraph A.6 of the Auxiliary Training Handbook - Boat Crew:
   - Perform a Navigation and Piloting Exercise / TASK BCM-08-02-AUX
   - Man Overboard
   - Assist the Coxswain with a Pre-Check off of an Auxiliary Facility
   - Towing Astern
   - Towing Alongside
   - Boat Handling
   - Assist in anchoring and weighing the Boat's Anchor
   - Demonstrate proficiency in knot tying and line handling
   - Demonstrate procedures to be followed in the event of a fire
   - Demonstrate procedures to be followed in the event of a grounding or striking of a submerged object

Remember to submit documentation of completed currency maintenance hours and tasks by December 31st of each calendar year in order to maintain certification and avoid being placed into Required Yearly Requirement (REYR) status.<br><br>

#### **Source Documents:**

*Auxiliary_Training_Handbook_Boat_Crew_ATH_16794*, page 44<br>

*2023 Surface Operations_Workshop Rev1*, page 25<br>

*Auxiliary_Training_Handbook_Boat_Crew_ATH_16794*, page 78<br>

*2023 Surface Workshop Rev1*, page 35<br>

*2023 Surface Workshop Rev1*, page 59<br>

*Auxiliary_Training_Handbook_Boat_Crew_ATH_16794*, page 45<br>

*Auxiliary_Training_Handbook_Boat_Crew_ATH_16794*, page 47<br>

*Auxiliary_Training_Handbook_Boat_Crew_ATH_16794*, page 43<br>


In [44]:
from IPython.display import display, Markdown


def create_long_source_list(response):
    '''Extracts a list of sources along with full source

    The dictionary has three elements (query, response, and source_documents). 
    Inside the third is a list with a custom object Document 
    associated with the key 'source_documents'
    '''

    markdown_list = []

    for i, doc in enumerate(response['source_documents'], start=1):
        page_content = doc.page_content
        source = doc.metadata['source']
        short_source = source.split('/')[-1].split('.')[0]
        page = doc.metadata['page']
        markdown_list.append(
            f"**Reference {i}:**    *{short_source}*, page {page}<br>  {page_content}\n")

    long_source_list = '\n'.join(markdown_list)
    return long_source_list


long_source_list = create_long_source_list(response)


# display list
display(Markdown("---"))
display(Markdown(f"#### **Full Source References:**"))
display(Markdown(long_source_list))
display(Markdown("---"))
display(Markdown("**Disclaimer:** This service only contains national documents. It is for informational use only and is not intended as a substitute for official policy.*"))

---

#### **Full Source References:**

**Reference 1:**    *Auxiliary_Training_Handbook_Boat_Crew_ATH_16794*, page 44<br>  Auxiliary Training Handbook – Boat Crew  
Chapter 5 –  Currency Maintenance 
 
 
5-2 
Section A.  Currency Maintenance 
Introduction  This section discusses the minimum currency requirements for 
maintaining certifications.  
In this Section  This section contains the following information:  
Title  Page  
General  5-2 
Currency Maintenance Cycle  5-2 
Crewmember  5-3 
Coxswain  5-4 
PWC Operator  5-5 
Currency Maintenance Documentation Requirement  5-5 
Nighttime Defintion  5-5 
TCT/RM Training  5-6 
Navigation Rules Exam  5-6 
Operational Workshops  5-7 
Documentation of Training  5-7 
 
  
General  Currency requirements consist of a set of tasks that must be performed 
every year along with annual underway hour requirements. 
  
Currency Maintenance Cycle  Currency maintenance is conducted on a three -year cycle, with certain 
requirements every year during the cycle, and requires the services of a Qualification Examiner (QE).  
The currency cycle begins on 01 January of the year following initial 
certification. Currency requirements must be met by 31 December of each year.   
For example,  if a member is certified as a coxswain on 15 July 2020, that 
member's first currency year begins  on 01 January 2021, and the member 
must meet all annual currency requirements by the end of 2021 (31 
December 2021). The third- year currency requirements must be met by 
31 December 202 3.  
Annual currency requirements must be met during the first full cale ndar 
year after certification. Credit will not be given to hours or tasks 
completed in the partial year of initial certification. Failure to meet 
currency requirements in any year of the cycle will cause a member's

**Reference 2:**    *2023 Surface Operations_Workshop Rev1*, page 25<br>  *** UPDATE***   Annual Training for 2023 
Currency requirements consist of a set of tasks that must be 
performed every year along with annual underway hour 
requirements. 
     Reference Auxiliary Training Handbook – Boat Crew (ATH-BC) -      
Chapter 5 for detailed qualification, certification, annual 
currency maintenance, and 3-year QE requirements for crew 
member, coxswain, and PWC operator. 
     Detailed reporting and tracking procedures will be released via 
     ALAUX in early 2023. 
Response Division – 2023 Surface 
Workshop U.S COAST GUARD AUXILIARY - UNCLASSIFIED 26

**Reference 3:**    *Auxiliary_Training_Handbook_Boat_Crew_ATH_16794*, page 78<br>  Auxiliary Training Handbook – Boat Crew  
Enclosure (4)  
 
 
2 
Member ID:   Task Currency Calendar Year:   
Persons authorized to sign off on Tasks completed shall record their name, signature, and initials in the table below.  
Sign Off Name  Sign Off Signature  Sign Off  
Initials  
   
   
   
   
   
ROLLUP TASK NAMES IN AUXDATA II  
The Annual Currency Maintenance Tasks shall be recorded in AUXDATA II as the rollup Tasks listed below, 
acknowledging all Tasks within a requirement section are complete:  
• (BCM) ANNUAL DAY TASKS  
• (BCM) ANNUAL NIGHT TASKS  
• (BCM) ANNUAL NIGHT U/W HOURS  
ROLLUP TASK DATES  IN AUXDATA II  
When all Annual  Currency Maintenance Tasks within a requirement section are completed within the designated Task 
Currency Calendar Year, the Task completion date for the rollup Task shall be recorded in AUXDATA II as the latest 
date lis ted in the corresponding requirement section.  
If one or more Currency Maintenance Tasks are completed for a requirement section after the designated Task Currency Calendar Year, the Task completion date for the rollup Task shall be recorded in AUXDATA II a s December 
31
st of the Task Currency Calendar Year listed on this form regardless of the latest date listed in the corresponding 
requirement section. Example:  
• Task Currency Calendar Year = 2023  
• One or more Tasks are completed during Calendar Year 2023, but  the final Task for a requirement section 
is completed on 5/25/2024.  
• The completion date to be recorded for the rollup Task = 12/31/2023  
FAILS TO MEET ANNUAL CURRENCY REQUIREMENTS  (ATH 16794.51 Ch. 4, Section C)   
When a member fails to meet annual currency requirements, their certification will lapse, and they will be placed in Required Yearly Requirement (REYR) status. A member whose certification has lapsed may participate as a 
designated trainee on an ordered pa trol. A member who fails to meet annual currency requirements for the year shall 
make up the missing hours and/or currency maintenance tasks (listed in the Task Sections on Page 1)  as a trainee, 
under the supervision of a certified coxswain the following c alendar year.   
Coxswain shall document completion of all missing hours and/or annual currency requirements  utilizing the Task 
Sections on Page 1. Upon completion of the missing task or hours, this may serve as the form al letter from the F C to 
the OTO documenting completion.  
1. The member has completed the missing requirement and (2) request that the member be re- instated.  
Position:  Name: (print)  Signature:  Date:  
Trainee:     
Coxswain:     
FC    
OTO:     
Members should keep a copy of the form for their records  

**Reference 4:**    *2023 Surface Workshop Rev1*, page 35<br>  Questions & Answers About the Policies 
Night Qualification : All certified crew or coxn certified prior to 01 Jan 
2023 will be grandfathered as night operations certified. If a crew/coxn 
wishes to maintain their night certification they will need to meet the 
annual currency maintenance requirements for night certification prior to 
31 Dec 2023. Members who fail to meet these minimum requirements 
will be placed into REYR for NIGHT operations only. Meeting all other 
annual currency requirements will maintain the crew/coxn certification 
during daytime operations.  
Response Division – 2023 Surface 
Workshop U.S COAST GUARD AUXILIARY - UNCLASSIFIED 36


---

**Disclaimer:** This service only contains national documents. It is for informational use only and is not intended as a substitute for official policy.*

## 7. Evaluate the model's performance


In [ ]:
# %pip install --upgrade tiktoken

In [ ]:
#  Get wandb Metrics-- WORKS ONLY IN JUPYTER NOTEBOOKS

# %wandb wks_consulting/ChatUSCG_notebook # Display a project workspace

# %wandb wks_consulting/ChatUSCG_notebook/runs/RUN_ID  # Display a single run

# %wandb wks_consulting/ChatUSCG_notebook/sweeps/SWEEP_ID # Display a sweep

# %wandb wks_consulting/ChatUSCG_notebook/reports/REPORT_ID # Display a report

# %wandb wks_consulting/ChatUSCG_notebook -h 2048 # Specify the height of embedded iframe

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model(config["model"])
query_tokens = encoding.encode(response['query'])
query_length = len(query_tokens)
source_tokens = encoding.encode(str(response['source_documents']))
source_length = len(source_tokens)
result_tokens = encoding.encode(response['result'])
result_length = len(result_tokens)
tokens = encoding.encode(str(response))
tot_length = len(tokens)


print(f"""
    Encoding: {encoding}

    {query_length} query
    {source_length} source
    {result_length} result
    {tot_length} Total tokens used

    GPT-3.5-turbo supports a context window of 4096 tokens
    GPT-3.5-turbo-16k supports a context window of 16,385 tokens
    GPT-4 supports a context window of 8192 tokens
    GPT-4-32k supports a context window of 32,768 tokens
""")

In [ ]:
import tiktoken


def count_tokens(response):
    ''' counts the tokens from the response'''
    encoding = tiktoken.encoding_for_model(config["model"])
    query_tokens = encoding.encode(response['query'])
    query_length = len(query_tokens)
    source_tokens = encoding.encode(str(response['source_documents']))
    source_length = len(source_tokens)
    result_tokens = encoding.encode(response['result'])
    result_length = len(result_tokens)
    tokens = encoding.encode(str(response))
    tot_length = len(tokens)

    return query_length, source_length, result_length, tot_length


# Usage:
response = {
    'query': "your_query_here",
    'source_documents': "your_source_documents_here",
    'result': "your_result_here"
}

query_len, source_len, result_len, total_len = count_tokens(response)

'''# use this one in python script
wandb.log({"tokens_used": tot_length, "16k context window": "4096 tokens"})
wandb.finish()  # this is only needed for the juypter notebook'''

In [ ]:
print(f"""
    Encoding: {encoding}

    {query_length} query
    {source_length} source
    {result_length} result
    {tot_length} Total tokens used

    GPT-3.5-turbo supports a context window of 4096 tokens
""")

Write response to a pickle file, overwriting existing pickle file

In [ ]:
# Save the response (a python dictionary) to a file
import pickle

with open("dummy_response.pkl", "wb") as file:
    pickle.dump(response, file)